# Imports

In [1]:
import os
import pathlib
import random
from math import sqrt

import pandas as pd
import plotly.express as px

PATH = pathlib.Path.cwd() / "src"

# set current working directory to the path
os.chdir(PATH)
print(f"Current path: {PATH}")

Current path: /Users/eliebrosset/Documents/GitHub/cymoo/src


In [2]:
from MOO import MOO
from optimizers.NSGA2 import NSGA2
from optimizers.NSRA import NSRA
from optimizers.NSWGE import NSWGE
from problems.multitask_routing.Network import Network
from problems.multitask_routing.Task import Task
from problems.Solution import Solution

# Defining the parameters

## Layers

In [3]:
# PROBLEM DEFINITION
paramsLayerOne = {
    "unit": {
        "tag": "DEVICE",
        "computingSpeed": lambda: random.randint(5, 10),
        "positionX": 0,
        "positionY": 0,
        "throughput": 0.5,
        "pollution": lambda: round(random.random(), 2),
        "cost": lambda: round(random.random(), 2),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 1,
        "flowRate": lambda: 1,
    },
    "numberNewUnits": 10,
}

paramsLayerTwo = {
    "unit": {
        "tag": "FOG",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(5, 10),
        "positionX": lambda x: x.positionX + random.randint(-2, 2),
        "positionY": lambda x: x.positionY + random.randint(-2, 2),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 2
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 20,
}

paramsLayerThree = {
    "unit": {
        "tag": "CLOUD",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(50, 100),
        "positionX": lambda x: x.positionX + random.randint(-5, 5),
        "positionY": lambda x: x.positionY + random.randint(-5, 5),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 3
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 10,
}

## Tasks

In [4]:
tasks = tuple(
    Task(random.randint(1, 1000), random.randint(1, 1000)) for _ in range(3)
)

## Network

In [5]:
problem = Network(
    "DEVICE",
    tasks=tasks,
    optimDirections={
        "processingTime": "min",
        "cost": "min",
        "pollution": "min",
    },
    minDepth=10,
    maxDepth=20,
    mutationRate=0.1,
    layers=[paramsLayerOne, paramsLayerTwo, paramsLayerTwo, paramsLayerThree],
)

## Optimization

In [6]:
nIterations = 10
nSolutions = 1000
seed = 10

# Optimization

## Training

In [7]:
moo = MOO(problem)

nsga2_paretos = moo.optimize(
    NSGA2,
    nSolutions,
    nIterations,
    saveDir=PATH.parent / "imgs",
    seed=seed,
    ratioKept=0.5,
)
nsra_paretos = moo.optimize(
    NSRA,
    nSolutions,
    nIterations,
    saveDir=PATH.parent / "imgs",
    seed=seed,
    ratioKept=0.5,
)
# nswge_paretos = moo.optimize(NSWGE, nSolutions, nIterations, seed=seed)


# DISPLAYING RESULTS



Optimizing with NSGA2...


100%|██████████| 10/10 [00:06<00:00,  1.64it/s]


Displaying pareto solutions...
{'processingTime': 8679.833244580217, 'cost': 24516.67828067495, 'pollution': 36924.50144683291}
{'processingTime': 8447.944355136373, 'cost': 39070.14771199331, 'pollution': 33784.202915709604}
{'processingTime': 9081.157473028357, 'cost': 25894.28676059864, 'pollution': 16789.403164833264}
{'processingTime': 10411.86514897959, 'cost': 18556.378425625993, 'pollution': 25313.083363493617}

Optimizing with NSRA...


100%|██████████| 10/10 [00:30<00:00,  3.10s/it]


Displaying pareto solutions...
{'processingTime': 9365.324567889296, 'cost': 36716.12920436481, 'pollution': 44905.91327038176}
{'processingTime': 9783.742751042715, 'cost': 52364.48953631066, 'pollution': 42380.025793729874}
{'processingTime': 14943.186280544, 'cost': 44111.27883962127, 'pollution': 29537.197198698996}
{'processingTime': 11017.49032935997, 'cost': 52483.34397124067, 'pollution': 32616.480868953506}
{'processingTime': 15624.373679734124, 'cost': 32037.15318905766, 'pollution': 31477.765477698245}
{'processingTime': 10028.74297591608, 'cost': 72032.71408759702, 'pollution': 38799.62726444011}
{'processingTime': 10320.43855125583, 'cost': 37938.912752015116, 'pollution': 34766.00304418073}
{'processingTime': 14423.849565022872, 'cost': 38608.31421602702, 'pollution': 30060.17838704982}
{'processingTime': 12891.431276090609, 'cost': 36802.222584118135, 'pollution': 43137.44431738177}
{'processingTime': 14037.927291736933, 'cost': 35243.520812867195, 'pollution': 36300.648

## Evaluation

In [8]:
MOO.relative_efficiency(
    nsra_paretos, nsga2_paretos, Solution.optimDirections, verbose=True
)

Relative efficiency: 0 % of <class 'optimizers.NSRA.NSRA'> solutions are undominated by <class 'optimizers.NSGA2.NSGA2'> solutions
Relative efficiency: 100 % of <class 'optimizers.NSGA2.NSGA2'> solutions are undominated by <class 'optimizers.NSRA.NSRA'> solutions


(0, 1)

# Results

In [9]:
def generate_violin_plot(df, title):
    fig = px.violin(
        df,
        x="Epoch",
        y="Value",
        box=True,
        hover_data=df.columns,
        template="plotly_white",
        color_discrete_sequence=["#1f77b4"],
    )
    fig.update_layout(
        title=title,
        xaxis_title="Epoch",
        yaxis_title="Value",
        font=dict(
            family="Times New Roman",
            size=12,
        ),
    )
    fig.show()


# generate random values for the pareto solutions for each epoch
def generate_random_values(epochs: int = 20):
    values = []
    for i in range(1, epochs + 1):
        for j in range(0, 10):
            values.append([i, random.randint(1, 100)])
    return values


# plot the violin
def plot_violin(epochs):
    values = generate_random_values(epochs)
    df = pd.DataFrame(values, columns=["Epoch", "Value"])
    # keep only 10 epochs depending on the number of epochs
    df = df[df["Epoch"] % (epochs // 10) == 0]
    generate_violin_plot(
        df, "Violin plot of the pareto solutions for each epoch (10 epochs)"
    )


plot_violin(50)
